# Table of Contents
 <p>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Citation network of drafts of Chinese constitution

Hive Plot visualization

# This is where the latest routines are...

We want to use [graph_tool](https://graph-tool.skewed.de/). This requires shell access and [docker](https://www.docker.com/) installed. If you have that, execute the following in a shell:

The last two lines should in fact be one single line and you should replace <my_working_dir> with the path to where you have this notebook stored. There, network graphs will be created as well. The last command should provide you with another shell in the now-running docker environment. In there, execute:

note down token, load <http://localhost:8888/?token=XY> in browser with XY being replaced by the token you have noted down. In the browser, a selection of files should appear, where you should click on the file "Fupeng_Li.ipynb", and the you end up in this file, in an environment where you should be able to execute the code cells below. (Just click inside and press Ctrl+Return.)

In [1]:
from graph_tool.all import *
import csv

hy = Graph()
v_name = hy.new_vertex_property("string")
v_color = hy.new_vertex_property("string")
v_size = hy.new_vertex_property("float")
# v_text_offset = hy.new_vertex_property('vector<double>')

# The file nodes.tsv contains the hierarchy of nodes, from a single "Origin" origin, in "from"-"to" relations
with open('nodes.tsv') as csvfile: 
    readCSV = csv.reader(csvfile, delimiter='\t')
    next(readCSV, None)  # skip the headers

    for row in readCSV:
        q  = row[0]       # from
        qc = row[1]       # from_color
        z  = row[2]       # to
        zc = row[3]       # to_color
        if len(graph_tool.util.find_vertex(hy, v_name, q)) == 0:
            s = hy.add_vertex()
            v_name[s] = q
            v_color[s] = qc
            # v_text_offset[s] = [0.0, 0.0]
            if qc == "black": v_size[s] = 0
            else: v_size[s] = 10
        else:
            s = graph_tool.util.find_vertex(hy, v_name, q)[0]

        if len(graph_tool.util.find_vertex(hy, v_name, z)) == 0:
            t = hy.add_vertex()
            v_name[t] = z
            v_color[t] = zc
            # v_text_offset[s] = [0.0, 0.0]
            if zc == "black": v_size[t] = 0
            else: v_size[t] = 10
        else:
            t = graph_tool.util.find_vertex(hy, v_name, z)[0]
        
        if not hy.edge(s, t):
            hy.add_edge(s, t)

pos = graph_tool.draw.radial_tree_layout(hy, hy.vertex(0), rel_order=v_name, weighted=True)

graph_draw(hy, pos=pos, vertex_fill_color=v_color,                                    # vertex drawing
           vertex_text=v_name, vertex_font_size=10, vertex_text_position="centered",  # vertex text
           output_size=(1900, 1600), fit_view=0.9, edge_pen_width=2, output="hierarchy.png")

if graph_tool.util.find_vertex(hy, "in", 0):
    l = graph_tool.util.find_vertex(hy, "in", 0)
    for n in l:
        print("Possible indegree problem with vertex {}".format(v_name[n]))

hy.save("hierarchy.xml.gz")

Possible indegree problem with vertex Origin


In [2]:
from matplotlib import colors

nw = Graph()
nwv_name     = nw.new_vertex_property("string")
nwv_color    = nw.new_vertex_property("string")
nwv_size_1   = nw.new_vertex_property("float")
nwv_size_2   = nw.new_vertex_property("float")
nwv_fontsize = nw.new_vertex_property("float")
nwv_shape    = nw.new_vertex_property("string")
nwe_color    = nw.new_edge_property("string")
nwe_gradient = nw.new_edge_property('vector<double>')
# we want these for masking later:
nwv_1923     = nw.new_vertex_property("bool")
nwv_1936     = nw.new_vertex_property("bool")
nwv_1923_2   = nw.new_vertex_property("bool")
nwv_1936_128 = nw.new_vertex_property("bool")
nwe_1923     = nw.new_edge_property("bool")
nwe_1936     = nw.new_edge_property("bool")
nwe_1923_2   = nw.new_edge_property("bool")
nwe_1936_128 = nw.new_edge_property("bool")


# g.edge_properties['egradient'] = egradient

for v in hy.vertices():
    n = nw.add_vertex()
    nwv_name[n]  = v_name[v]
    nwv_color[n] = v_color[v]
    nwv_size_1[n]  = v_size[v]
    nwv_size_2[n]  = 3
    
    if nwv_name[n][:10] == "Draft 1923":
        nwv_1923[n] = True
    else:
        nwv_1923[n] = False

    if nwv_name[n][:10] == "Draft 1936":
        nwv_1936[n] = True
    else:
        nwv_1936[n] = False
        
    if nwv_name[n] == "Draft 1923_Art.2 (1)":
        nwv_1923_2[n] = True
    else:
        nwv_1923_2[n] = False
    
    if nwv_name[n] == "Draft 1936_Art.128":
        nwv_1936_128[n] = True
    else:
        nwv_1936_128[n] = False

    if nwv_name[n][:5] == "Draft":
        nwv_size_1[n] = 20
        nwv_fontsize[n] = 15       
    else:
        nwv_fontsize[n] = 12
        

with open('edges.tsv') as csvfile:
    count = 0
    readCSV = csv.reader(csvfile, delimiter = '\t')
    next(readCSV, None)  # skip the headers

    for row in readCSV:
        source = ""
        target = ""
        
        f = row[0]       # from
        t = row[1]       # to
    
        if len(graph_tool.util.find_vertex(nw, nwv_name, f)) == 0:
            print("Node hierarchy does not contain '{}', although it occurs as a source in edges list.".format(f))
            break
        else:
            source = graph_tool.util.find_vertex(nw, nwv_name, f)[0]
        
        if len(graph_tool.util.find_vertex(nw, nwv_name, t)) == 0:
            print("Node hierarchy does not contain '{}', although it occurs as a target in edges list.".format(t))
            break
        else:
            target = graph_tool.util.find_vertex(nw, nwv_name, t)[0]
        
        e = nw.add_edge(source, target)
        count += 1
        nwe_color[e] = nwv_color[source]
        estartcolor = [0] + list(colors.to_rgba(nwv_color[target]))
        eendcolor = [1] + list(colors.to_rgba(nwv_color[source]))
        nwe_gradient[e] = estartcolor + eendcolor
        
        print(str(count) + ": Create edge from '" + f + "' to '" + t + "'.")

        if nwv_name[source][:10] == "Draft 1923":
            nwv_1923[target] = True
            nwe_1923[e]      = True
        else:
            nwv_1923[n] = False
            nwe_1923[e] = False

        if nwv_name[source][:10] == "Draft 1936":
            nwv_1936[target] = True
            nwe_1936[e]      = True
        else:
            nwv_1936[n] = False
            nwe_1936[e] = False

        if nwv_name[source] == "Draft 1923_Art.2 (1)":
            nwv_1923_2[target] = True
            nwe_1923_2[e]      = True
        else:
            nwv_1923_2[n] = False
            nwe_1923_2[e] = False

        if nwv_name[source] ==  "Draft 1936_Art.128":
            nwv_1936_128[target] = True
            nwe_1936_128[e]      = True
        else:
            nwv_1936_128[n] = False
            nwe_1936_128[e] = False


l = graph_tool.util.find_vertex(nw, "total", 0)
for i, n in enumerate(l):
    print(str(i+1) + ": 0-degree vertex: {}".format(v_name[n]))

nw.save("network.xml.gz")

graph_draw(nw, vertex_fill_color=nwv_color, vertex_size=nwv_size_1,
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position=10,
           output_size=(1900, 1600), fit_view=0.9, edge_pen_width=2, output="network_1.png")


1: Create edge from 'Draft 1923_Art.1' to '1919 Germany_Art.151'.
2: Create edge from 'Draft 1923_Art.2 (1)' to 'Book of Mencius'.
3: Create edge from 'Draft 1923_Art.2 (1)' to '1920 Czechoslovakia_Art.108'.
4: Create edge from 'Draft 1923_Art.2 (1)' to '1920 Czechoslovakia_Art.109'.
5: Create edge from 'Draft 1923_Art.2 (1)' to '1920 Far Eastern Republic/Chita'.
6: Create edge from 'Draft 1923_Art.2 (1)' to '1921 Poland_Art.99'.
7: Create edge from 'Draft 1923_Art.2 (1)' to '1911 Portugal_Art.25'.
8: Create edge from 'Draft 1923_Art.2 (1)' to 'Pre-Qin Scholars'.
9: Create edge from 'Draft 1923_Art.2 (1)' to '1918 Soviet Russia'.
10: Create edge from 'Draft 1923_Art.2 (1)' to '1919 Germany_Art.153'.
11: Create edge from 'Draft 1923_Art.2 (1)' to '1919 Germany_Art.155 (1)'.
12: Create edge from 'Draft 1923_Art.2 (1)' to '1919 Germany_Art.155 (2)'.
13: Create edge from 'Draft 1923_Art.2 (1)' to '1919 Germany_Art.155 (3)'.
14: Create edge from 'Draft 1923_Art.2 (2)' to '1921 Poland_Art.99

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x7fd841549128, at 0x7fd8415499b0>

Now, create an edge-bundling visualization according to:

Holten, Danny: "Hierarchical Edge Bundles: Visualization of Adjacency Relations in Hierarchical Data", in: *IEEE Transactions on Visualization and Computer Graphics* 12/5 (2006), pp. 741–748, DOI: 10.1109/TVCG.2006.147.

In [3]:
# state = graph_tool.inference.minimize.minimize_nested_blockmodel_dl(nw, deg_corr=True)
# t     = graph_tool.inference.nested_blockmodel.get_hierarchy_tree(state)[0]

hypos  = graph_tool.draw.radial_tree_layout(hy, hy.vertex(0), rel_order=v_name, weighted=True)

cts08   = graph_tool.draw.get_hierarchy_control_points(nw, hy, hypos, beta=0.8, max_depth=3)
cts06   = graph_tool.draw.get_hierarchy_control_points(nw, hy, hypos, beta=0.6, max_depth=3)
pos   = nw.own_property(hypos)

l = graph_tool.util.find_vertex(nw, "total", 0)
m = [n for n in l if v_name[n]!="Chinese Traditions" and v_name[n]!="Party's Doctrines"]
for p in m:
    print("Removing 0-degree vertex {}".format(v_name[p]))
nw.remove_vertex(m)
# ============== Output for both Drafts ==============

# output with beta=0.6, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                  # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered", # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_gradient=nwe_gradient,              # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="both_b06_gradient.png")

# output with beta=0.8, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="both_b08_gradient.png")


# output with beta=0.6, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="both_b06_solid.png")

# output with beta=0.8, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="both_b08_solid.png")

## ============ Only Draft 1923 ===============
nw.set_vertex_filter(nwv_1923)
nw.set_edge_filter(nwe_1923)

# output with beta=0.6, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1923_b06_gradient.png")

# output with beta=0.8, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1923_b08_gradient.png")


# output with beta=0.6, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1923_b06_solid.png")

# output with beta=0.8, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1923_b08_solid.png")

nw.set_vertex_filter(None)
nw.set_edge_filter(None)

## ============ Only Draft 1936 ===============
nw.set_vertex_filter(nwv_1936)
nw.set_edge_filter(nwe_1936)

# output with beta=0.6, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1936_b06_gradient.png")

# output with beta=0.8, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1936_b08_gradient.png")


# output with beta=0.6, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1936_b06_solid.png")

# output with beta=0.8, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1936_b08_solid.png")


nw.set_vertex_filter(None)
nw.set_edge_filter(None)

## ============ Only Draft 1923, Art. 2 (1) ===============
nw.set_vertex_filter(nwv_1923_2)
nw.set_edge_filter(nwe_1923_2)

# output with beta=0.6, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1923-2_b06_gradient.png")

# output with beta=0.8, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1923-2_b08_gradient.png")


# output with beta=0.6, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1923-2_b06_solid.png")

# output with beta=0.8, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1923-2_b08_solid.png")


nw.set_vertex_filter(None)
nw.set_edge_filter(None)

## ============ Only Draft 1936, Art. 128   ===============
nw.set_vertex_filter(nwv_1936_128)
nw.set_edge_filter(nwe_1936_128)

# output with beta=0.6, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1936-128_b06_gradient.png")

# output with beta=0.8, gradient-colored edges and vertices shown
graph_draw(nw, inline=True, pos=pos, 
           vertex_fill_color=nwv_color, vertex_size=nwv_size_1,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_gradient=nwe_gradient,            # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1936-128_b08_gradient.png")


# output with beta=0.6, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts06, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1936-128_b06_solid.png")

# output with beta=0.8, solid-colored edges and vertices hidden
graph_draw(nw, inline=True, pos=pos,
           vertex_shape="none",
           vertex_fill_color=nwv_color, vertex_size=nwv_size_2,                                # vertex drawing
           vertex_text=nwv_name, vertex_font_size=nwv_fontsize, vertex_text_position="centered",                              # vertex text
           edge_control_points=cts08, edge_pen_width=1, edge_color=nwe_color,                  # edge drawing
           output_size=(1900, 1600), fit_view=0.9, output="1936-128_b08_solid.png")


nw.set_vertex_filter(None)
nw.set_edge_filter(None)

print("Done.")

Removing 0-degree vertex Origin
Removing 0-degree vertex Draft Articles
Removing 0-degree vertex Foreign Legislation
Removing 0-degree vertex Chinese Sources
Removing 0-degree vertex 1850 CM
Removing 0-degree vertex 1874 Switzerland
Removing 0-degree vertex 1911 Portugal
Removing 0-degree vertex 1917 Mexico
Removing 0-degree vertex 1919 TV
Removing 0-degree vertex 1920 Austria
Removing 0-degree vertex 1920 Czechoslovakia
Removing 0-degree vertex 1920 Estonia
Removing 0-degree vertex 1920 FER
Removing 0-degree vertex 1920 League of Nations
Removing 0-degree vertex 1921 Poland
Removing 0-degree vertex 1922 Netherlands
Removing 0-degree vertex 1923 Romania
Removing 0-degree vertex 1924 Honduras
Removing 0-degree vertex 1928 Lithuania
Removing 0-degree vertex 1929 Ecuador
Removing 0-degree vertex 1931 Spain
Removing 0-degree vertex 1931 Yugoslavia
Removing 0-degree vertex 1933 Peru
Removing 0-degree vertex 1933 Portugal
Removing 0-degree vertex 1923 China Draft
Removing 0-degree vertex 193